In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean          SD         Naive SE       MCSE         ESS    
         lp__ -39.388360475  3.271433369 0.05172590330 0.1565005161  436.962945
accept_stat__   0.947415335  0.074537773 0.00117854567 0.0012123206 1000.000000
   stepsize__   0.046491575  0.004532452 0.00007166436 0.0007256825   39.009752
  treedepth__   5.889750000  0.477652568 0.00755235022 0.0225618452  448.203252
 n_leapfrog__  73.082000000 33.942912461 0.53668456898 1.0251953660 1000.000000
  divergent__   0.000000000  0.000000000 0.00000000000 0.0000000000         NaN
     energy__  45.901920350  4.203963502 0.06647049933 0.1786827132  553.544828
        alpha   1.118162844  0.716208365 0.01132424857 0.0224790355 1000.000000
  a_society.1  -0.203898219  0.241319429 0.00381559519 0.0057583366 1000.000000
  a_society.2   0.042056350  0.221176150 0.00349710199 0.0059298995 1000.000000
  a_

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*